In [ ]:
import awswrangler as wr
import pandas as pd

In [ ]:
dfOriginal = wr.s3.read_parquet(f"s3://cchc-dw-qa-staging/monday/proyectos/", dataset=True)

In [ ]:
df = dfOriginal.copy()
keep_out = [
        "Antecedentes",
        "Antecedentes de Proyecto",
        "Antecedentes de Proyectos",
        "Antecedentes del Proyecto",
        "Cabecera",
        ]

print(f"cantidad antes de quitar hitos irrelevantes : {len(df)}")
df = df[~df.nombre_hito.isin(keep_out)]
print(f"cantidad después de quitar hitos irrelevantes : {len(df)}")

df["id_proyecto"] = df.id_proyecto + df.id_tablero

id_proyectos = df.id_proyecto.drop_duplicates().to_list()
dfAgrupado = df.groupby(by='id_proyecto', as_index=False).agg({'fecha_planificada_inicio': "min", "fecha_planificada_fin": "max" , 'fecha_real_inicio': "min", "fecha_real_fin": "max" })


for id in id_proyectos:
        df2 = df.copy()
        df2 = df2[df2.id_proyecto == id]
        df2Agrupado = dfAgrupado[dfAgrupado.id_proyecto == id]
        dfFinalizado = df2[df2.is_finalizado == False]
        df2 = df2.iloc[0]
        df2 = df2.to_frame().T
    
        if len(dfFinalizado) > 0:
                df2['is_finalizado'] = bool(False)
        else: 
                df2['is_finalizado'] = bool(True)
                
        df2['nombre_hito'] = 'H0: Fecha Proyectos'
        df2['fecha_planificada_inicio'] = df2Agrupado.iloc[0].fecha_planificada_inicio
        df2['fecha_planificada_fin'] = df2Agrupado.iloc[0].fecha_planificada_fin
        df2['fecha_real_inicio'] = df2Agrupado.iloc[0].fecha_real_inicio
        df2['fecha_real_fin'] = df2Agrupado.iloc[0].fecha_real_fin
        df2['nombre_lider_hito'] = None
        df2['gerencia_hito'] = None
        df2['id_hito'] = None
        df = pd.concat([df ,df2], axis=0, ignore_index=True)

df

In [ ]:
response = wr.s3.to_parquet(
    df=df,
    path=f"s3://cchc-dw-qa-analytics/monday/proyectos/",
    dataset=True,
    table="monday_proyectos",
    database="analytics_qa",
    mode="overwrite",
    dtype={
        "fecha_planificada_inicio" : "date",
        "fecha_planificada_fin" : "date",
        "fecha_real_inicio" : "date",
        "fecha_real_fin" : "date"
    }
)